In [1]:
#!pip3 install turicreate
#pip install nltk
#conda install gensim worked

In [2]:
import re
import gensim
from gensim import corpora, models, similarities
import nltk
import pandas as pd
import os
import codecs
import gensim.models.word2vec as w2v
import time

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# to use multiple cores for heavy duty functions, but the installation did not work
# import turicreate

In [5]:
corpus = pd.read_csv("/home/ubuntu/datasets/cancer-clinical-trials-eligibility/labeledEligibilitySample1000000.csv", 
                     sep='\t',
                     error_bad_lines=False,
                     header=None)

In [6]:
len(corpus)

1000000

In [7]:
corpus.head()

,0,1
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [8]:
corpus.iloc[:,0].unique()

array(['__label__0', '__label__1'], dtype=object)

In [9]:
corpus.iloc[:,1].head()

0    study interventions are recombinant CD40-ligan...
1    study interventions are Liposomal doxorubicin ...
2    study interventions are BI 836909 . multiple m...
3    study interventions are Immunoglobulins . recu...
4    study interventions are Paclitaxel . stage ova...
Name: 1, dtype: object

In [10]:
corpus_data= pd.DataFrame(corpus.iloc[:,1])

In [11]:
corpus.shape, type(corpus)

((1000000, 2), pandas.core.frame.DataFrame)

In [12]:
corpus.columns = ['label','text']
corpus.head()

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [13]:
corpus_blob = corpus.groupby('label')['text'].apply(' '.join).reset_index()
corpus_blob

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__1,study interventions are Prednisone . patients ...


In [14]:
inclusion_criteria_0 = corpus_blob['text'][0]
len(inclusion_criteria_0),inclusion_criteria_0[0:500]

(83466072,
 'study interventions are recombinant CD40-ligand . melanoma skin diagnosis and no active cns metastases by ct scan or mri study interventions are Liposomal doxorubicin . colorectal cancer diagnosis and cardiovascular study interventions are BI 836909 . multiple myeloma diagnosis and indwelling central venous cateder or willingness to undergo intra venous central line placement study interventions are Immunoglobulins . recurrent fallopian tube carcinoma diagnosis and patients are allowed to receiv')

In [15]:
inclusion_criteria = corpus['text']

In [16]:
stoplist = set('for a of the and to in is at an must be with are but not no none has have other from as prior or except none see below . study , use " one two three four five six patients before start greater than any allowed by for they since'.split())
texts = [[word for word in inclusion_criteria.split() if word not in stoplist]
         for inclusion_criteria in inclusion_criteria]

In [17]:
len(texts)

1000000

In [18]:
texts[11:20]

[['interventions',
  'Androgens',
  'prostate',
  'cancer',
  'diagnosis',
  'patient',
  'characteristics'],
 ['interventions',
  'Bortezomib',
  'multiple',
  'myeloma',
  'diagnosis',
  'renal',
  'insufficiency',
  's'],
 ['interventions',
  'Prednisone',
  'testicular',
  'lymphoma',
  'diagnosis',
  'bulky',
  'disease',
  'greater_than',
  'equal_than',
  'cm',
  'least',
  'dimension'],
 ['interventions',
  'Razoxane',
  'unspecified',
  'adult',
  'solid',
  'tumor',
  'protocol',
  'specific',
  'diagnosis',
  'platelet',
  'count',
  'greater_than',
  'hundred',
  'zero',
  'mm³'],
 ['interventions',
  'Fludarabine',
  'natural_killer',
  'cell',
  'lymphoblastic',
  'leukemia',
  'lymphoma',
  'diagnosis',
  'bilirubin',
  'more',
  'doc',
  'mg',
  'dl',
  'alanine',
  'aminotransferase',
  'alt',
  'aspartate',
  'aminotransferase',
  'ast',
  'alkaline',
  'phosphatase',
  'alp',
  'more',
  'times',
  'upper',
  'limit',
  'normal',
  'uln'],
 ['interventions',
  'Fluda

In [19]:
len(texts), texts[1:10]

(1000000,
 [['interventions',
   'Liposomal',
   'doxorubicin',
   'colorectal',
   'cancer',
   'diagnosis',
   'cardiovascular'],
  ['interventions',
   'BI',
   '836909',
   'multiple',
   'myeloma',
   'diagnosis',
   'indwelling',
   'central',
   'venous',
   'cateder',
   'willingness',
   'undergo',
   'intra',
   'venous',
   'central',
   'line',
   'placement'],
  ['interventions',
   'Immunoglobulins',
   'recurrent',
   'fallopian',
   'tube',
   'carcinoma',
   'diagnosis',
   'receive',
   'required',
   'receive',
   'additional',
   'cytotoxic',
   'regimens',
   'management',
   'recurrent',
   'persistent',
   'disease',
   'more',
   'non',
   'platinum',
   'non',
   'taxane',
   'regimen'],
  ['interventions',
   'Paclitaxel',
   'stage',
   'ovarian',
   'cancer',
   'diagnosis',
   'recovered',
   'effects',
   'recent',
   'surgery',
   'radiotherapy',
   'therapy'],
  ['interventions',
   'Antibodies,',
   'Monoclonal',
   'recurrent',
   'verrucous',
   'carc

In [20]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
type(texts)

list

In [22]:
#f = codecs.open('C:\Python26\text.txt', 'r', 'utf-8-sig')
#text = f.read()
tok_corp = nltk.word_tokenize(str(texts))

In [23]:
len(tok_corp), tok_corp[1:20]

(49526498,
 ['[',
  "'interventions",
  "'",
  ',',
  "'recombinant",
  "'",
  ',',
  "'CD40-ligand",
  "'",
  ',',
  "'melanoma",
  "'",
  ',',
  "'skin",
  "'",
  ',',
  "'diagnosis",
  "'",
  ','])

#### Let's try with 14 workers

In [24]:
start = time.time()

ct2vec = w2v.Word2Vec(
    texts,
    workers=16,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

print ('word_tokenize():\t', time.time() - start)

word_tokenize():	 53.3589084148407


# Let's move on to inferences

In [25]:
print(ct2vec['aml'])

[-1.5130395   1.0651273  -1.3339647   1.9434758  -1.0667231  -3.723767
  0.42655203 -1.263188    1.0921146   0.35902965  1.0599599   0.3029279
 -1.6345255  -0.18858792 -0.02089053 -0.25254977  0.3574394   0.11491508
  3.8129048   2.176203    3.0383651   2.18803     2.1440363  -3.3317416
 -3.8390663  -0.13867578 -2.3765476   0.35668296  0.92299575  1.9025663
 -1.3185846   2.0637517  -2.3231142  -0.7959717  -2.0102618   1.4483111
  4.320182    0.91007864  0.19460995 -1.6897639   2.5530157  -0.26437318
 -0.3735732  -0.68136525  0.4627584  -3.3380866  -3.083154   -0.8690795
  0.625053   -4.056018   -0.6529801  -1.3461798  -1.0131177  -0.9501094
  0.07059646 -1.4425861  -0.03741329  0.15734145 -6.708997   -0.15194938
  1.7021779  -2.4321184  -1.6103584   0.8285542  -1.1081165  -0.6382768
  3.6999745   1.8984995  -0.84975266  1.2356646   0.7284954   1.2739803
 -0.60046846  3.0264533   3.748585    1.0606896  -1.861937    2.0555112
  0.1829372   3.1792302   0.88453233 -3.304588    0.45827127 -

^ This is how the actual embeddings look like for a word. Any concept is represented using a collection of 100 variables as we have specified it in word2vec function call above.

In [26]:
len(ct2vec['aml'])

100

In [27]:
ct2vec.most_similar('aml')

[('mds', 0.7235063910484314),
 ('crthree', 0.6261923313140869),
 ('cml', 0.5991892218589783),
 ('myelodysplasia', 0.5828728079795837),
 ('raeb', 0.5643167495727539),
 ('apl', 0.5623947978019714),
 ('blast', 0.5532690286636353),
 ('myeloblasts', 0.5460888147354126),
 ('atll', 0.5376821756362915),
 ('blasts', 0.5362112522125244)]

^ This is interesting as MDS is precursor to AML, MDS trial is often combined with AML.

In [28]:
ct2vec.most_similar('Sorafenib')

[('Niacinamide', 0.9934252500534058),
 ('Sunitinib', 0.9160200953483582),
 ('Bevacizumab', 0.8969194889068604),
 ('Tremelimumab', 0.8870413899421692),
 ('Everolimus', 0.8860596418380737),
 ('Sirolimus', 0.8782651424407959),
 ('Trebananib', 0.8664243221282959),
 ('Interferons', 0.8612068295478821),
 ('Dasatinib', 0.8541008830070496),
 ('Interferon-alpha', 0.8526219725608826)]

Let's try the blockbuster drug Keytruda

In [29]:
ct2vec.most_similar('Pembrolizumab')

[('Nivolumab', 0.94188392162323),
 ('Carboplatin', 0.8752785325050354),
 ('Tremelimumab', 0.8730031847953796),
 ('Everolimus', 0.8719791769981384),
 ('Vaccines', 0.871956467628479),
 ('Sirolimus', 0.8719024658203125),
 ('Azacitidine', 0.8702929615974426),
 ('Antibodies', 0.8689152002334595),
 ('Immunoglobulins', 0.8639222979545593),
 ('Bevacizumab', 0.8617658615112305)]

Pembrolizumab is a CheckPoint Inhibitor which targets PD-1/PD-L1. Interestingly we can see a few more CPIs in the results like Nivolumab, Tremelimumab. Additional, these CPIs are often combined with mAb like Bevacizumab or chemo like Carboplatin

Let's try Enzalutamide a drug used in Castrate-Resistant Prostate Cancer.

In [30]:
ct2vec.most_similar('Enzalutamide')

[('Hormones', 0.798703134059906),
 ('Antagonists', 0.7974197864532471),
 ('Androgens', 0.7825336456298828),
 ('Deslorelin', 0.7810865044593811),
 ('Goserelin', 0.773699939250946),
 ('Methyltestosterone', 0.7691765427589417),
 ('Denosumab', 0.7683835029602051),
 ('Diphosphonates', 0.7679646611213684),
 ('Estradiol', 0.7661968469619751),
 ('Foxy-5', 0.7647279500961304)]

These results are not that useful. Enzalutamide is a second generation anti-androgen drug (NAAD). I was expecting something like Apalutamide or Abiraterone.

In [31]:
ct2vec.most_similar('Estrogen')

[('Receptor', 0.9366602897644043),
 ('Modulators', 0.9226491451263428),
 ('Selective', 0.8971734642982483),
 ('Androgen', 0.8226393461227417),
 ('Endothelin', 0.7972637414932251),
 ('Female', 0.7724226713180542),
 ('Esterified', 0.7712032198905945),
 ('Antagonists', 0.759407639503479),
 ('TNBC', 0.7290056943893433),
 ('Leukotriene', 0.7271155118942261)]

^ This is again a moderately useful result. Estrogen is one of the hormone and Estrogen receptor plays a key role in a type of Breast Cancer, the other 2 being Progesteron and HER2 (both missing). When all three have negative status we call it a Triple-Negative Breast Cancer, which you can see above as TNBC. 

In [32]:
ct2vec.most_similar('platinum')

[('platin', 0.7030496001243591),
 ('taxane', 0.6441717743873596),
 ('cisplatin', 0.6221863031387329),
 ('carboplatin', 0.5843871831893921),
 ('docetaxel', 0.5569339990615845),
 ('irinotecan', 0.5461264848709106),
 ('paclitaxel', 0.5292472243309021),
 ('gemcitabine', 0.5221594572067261),
 ('front', 0.5139111280441284),
 ('anthracycline', 0.5077583193778992)]

Since platin-based chemotherapies are widely used in treating various types of cancer, we do see specific therapies like Carboplatin, Cisplatin in the results.

However, we missed Oxaliplatin. Also we see taxane-based chemotherapies

In [36]:
ct2vec.most_similar('taxane')

[('docetaxel', 0.7668116092681885),
 ('taxanes', 0.7520090341567993),
 ('gemcitabine', 0.7359477877616882),
 ('anthracycline', 0.6918892860412598),
 ('doublet', 0.670578122138977),
 ('carboplatin', 0.6650657653808594),
 ('trastuzumab', 0.6632235050201416),
 ('paclitaxel', 0.6628816723823547),
 ('vinorelbine', 0.6612695455551147),
 ('cisplatin', 0.6593953371047974)]

We do see docetaxane, paclitaxane in the results. Interestingly gemcitabine is often combined with taxanes in treating various types of tumors.

In [37]:
ct2vec.most_similar('CD20')

[('Adoptive', 0.9501046538352966),
 ('(CMV)-Specific', 0.9440615177154541),
 ('Cytomegalovirus', 0.9419352412223816),
 ('Regulatory', 0.9336546659469604),
 ('re-directed', 0.9330792427062988),
 ('Anti-CD19', 0.9325973987579346),
 ('AIJP', 0.9322923421859741),
 ('(Tcon)', 0.9313881993293762),
 ('sonographical', 0.9234209656715393),
 ('FAmily', 0.9212117195129395)]

In [38]:
ct2vec.most_similar('Anti-CD19')

[('Adoptive', 0.9669453501701355),
 ('re-directed', 0.9485307931900024),
 ('methoxyamine', 0.9469189643859863),
 ('Buspirone', 0.9443287253379822),
 ('orazdeight', 0.9441092014312744),
 ('Anti-CD20', 0.9436116218566895),
 ('wocpb', 0.94005286693573),
 ('Cytomegalovirus', 0.9396030902862549),
 ('mesenterial', 0.9366781711578369),
 ('fiveac', 0.9335872530937195)]

In [39]:
ct2vec.most_similar('Anti-CD20')

[('Adoptive', 0.9515570998191833),
 ('Anti-CD19', 0.9436115622520447),
 ('re-directed', 0.9316964149475098),
 ('CD20', 0.9195340871810913),
 ('Regulatory', 0.890544593334198),
 ('(CMV)-Specific', 0.8848384618759155),
 ('Cytomegalovirus', 0.881388783454895),
 ('(Tcon)', 0.8786172866821289),
 ('Retroviral', 0.8754958510398865),
 ('Passy', 0.8741341829299927)]

In [40]:
ct2vec.most_similar('rituximab')

[('brentuximab_vedotin', 0.6643871068954468),
 ('velcade', 0.6260936856269836),
 ('chop', 0.6114883422851562),
 ('bortezomib', 0.6056703925132751),
 ('bendamustine', 0.591356098651886),
 ('radioimmunotherapy', 0.5589418411254883),
 ('carfilzomib', 0.5559267401695251),
 ('cyclophosphamide', 0.544562816619873),
 ('alemtuzumab', 0.5433436036109924),
 ('monotherapy', 0.5314849019050598)]

Rituximab is a monoclonal anti-CD20 antibody used to treat non-Hodgkin's lymphoma, chronic lymphocytic leukemia, Wegener's granulomatosis, pemphigus vulgaris, and rheumatoid arthritis. Other anti-cd20 mAbs include Ofatumumab, Obinutuzumab, Ocrelizumab which we do not see here.